In [1]:
import sys
sys.path.insert(1, '../scr')
from toolbox import Configurator, AnkiCommunicator, AnkiCardWriter
from Collins import Collins_reader
from ExerciseWriter import ExerciseWriter
from Exercise import ExerciseFactory, ExerciseGatherer, ExampleSentences
import pyperclip
from pathlib import Path
import shutil
import json


def copy_folder(source_folder, destination_folder) -> None:
    destination_path = Path(destination_folder)
    source_path = Path(source_folder)
    if not destination_path.exists():
        shutil.copytree(source_path, destination_path)

### Initial set-up

In [2]:
# User inputs

date = 20240503
# category = 'Vocab Workout'
category = 'Vocab Explorer'
num_of_words_to_learn = 40

In [3]:
# Step 0: Create tmp file
tmp_dict = dict()
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file)


# Step 1: Get the words to learn for tomorrow
configurator = Configurator(json_path=f'../scr/config/{category}/config.json')
tomorrow_new_unabridged = configurator.get_n_words_to_learn(num_of_words_to_learn)
tmp_dict['tomorrow new unabridged'] = tomorrow_new_unabridged
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file, indent=4)



# Step 2: Add the new words to Anki

# Step 2.1: Look up the words in the dictionary
reader = Collins_reader(word_list=tomorrow_new_unabridged)
entry_list = reader.get_word_entry_list()


# Step 2.2: Export csv files and add the cards to Anki
writer = AnkiCardWriter(entry_list)
writer.write_cards(f'Exports/{date}/Anki Cards.csv')


# Step 3: Get the words to revisit tomorrow
anki = AnkiCommunicator()
tomorrow_review_unabridged = anki.get_words_for_tomorrow(deck_name=category, item='Front')
tmp_dict['tomorrow review unabridged'] = tomorrow_review_unabridged
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file, indent=4)


# Step 4: Shortlist the word lists
tomorrow_new = tomorrow_new_unabridged
tomorrow_review = tomorrow_review_unabridged
tomorrow_all = tomorrow_new + tomorrow_review
print(f'''
Number of all words: {len(tomorrow_all)};
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')
tmp_dict['tomorrow new'] = tomorrow_new
tmp_dict['tomorrow review'] = tomorrow_review
with open('tmp.json', 'w') as file:
    json.dump(tmp_dict, file, indent=4)


Number of all words: 73;
Number of new words: 40;
Number of old words: 33.



In [4]:
# Make a new folder
template_folder = '../scr/resources/Templates'
export_folder = f'./Exports/{date}'
copy_folder(template_folder, export_folder)

# Set up exercise writer 
def_template_path = '/Definition/template.tex'
def_completion_template_path = '/Definition_Completion/template.tex'
reading_template_path = '/Reading/template.tex'
gap_filling_template_path = '/Fill-in-the-gap/template.tex'
paraphrase_template_path = '/Paraphrase/template.tex'
def_output_folder = export_folder + '/Definition'
def_completion_output_folder = export_folder + '/Definition_completion'
reading_output_folder = export_folder + '/Reading'
gap_filling_output_folder = export_folder + '/Fill-in-the-gap'
paraphrase_output_folder = export_folder + '/Paraphrase'


factory = ExerciseFactory()
def_gatherer = ExerciseGatherer()
def_writer = ExerciseWriter(gatherer=def_gatherer, template_path=def_template_path, output_folder=def_output_folder)
def_completion_gatherer = ExerciseGatherer()
def_completion_writer = ExerciseWriter(gatherer=def_completion_gatherer, template_path=def_completion_template_path, output_folder=def_completion_output_folder)
reading_gatherer = ExerciseGatherer()
reading_exercise_writer = ExerciseWriter(gatherer=reading_gatherer, template_path=reading_template_path, output_folder=reading_output_folder)
gap_filling_gatherer = ExerciseGatherer()
gap_filling_exercise_writer = ExerciseWriter(gatherer=gap_filling_gatherer, template_path=gap_filling_template_path, output_folder=gap_filling_output_folder)
paraphrase_gatherer = ExerciseGatherer()
paraphrase_exercise_writer = ExerciseWriter(gatherer=paraphrase_gatherer, template_path=paraphrase_template_path, output_folder=paraphrase_output_folder)

### Definition

#### Set the word list and the set index here and get the prompt to create definitions

In [ ]:
def_start_idx = 11
def_end_idx = 20
def_set_index = 2

def_word_list = tomorrow_new[def_start_idx-1:def_end_idx]
definitions_set = factory.create_exercise(exercise_type='Definition', word_list=def_word_list)

#### Method 1: Get the definition from the dictionary

In [ ]:
definitions_set.import_definition_from_dictionary()
definitions_set.finish_import()
def_gatherer.import_exercise(definitions_set)
def_writer.render_template(set_index=def_set_index)

#### Method 2 (Obsolete): Create prompt and go to Chat GPT

In [ ]:
definitions_set.create_prompt()
definitions_set.get_prompt()

##### Paste the definitions here

In [ ]:
definitions = pyperclip.paste()
definitions_set.import_definition(text=definitions)
definitions_set.finish_import()
def_gatherer.import_exercise(definitions_set)
def_writer.render_template(set_index=def_set_index)

### Definition and Example Completion

#### Enter the index

In [ ]:
def_completion_start_idx = 16
def_completion_end_idx = 33
def_completion_set_index = 6

# def_completion_word_list = tomorrow_new[def_completion_start_idx-1:def_completion_end_idx]
def_completion_word_list = tomorrow_review[def_completion_start_idx-1:def_completion_end_idx]


definition_completion_exercise = factory.create_exercise(exercise_type='Definition and Example Completion', 
                                                         word_list=def_completion_word_list)
definition_completion_exercise.generate_exercise()
definition_completion_exercise.finish_import()
def_completion_gatherer.import_exercise(definition_completion_exercise)
def_completion_writer.render_template(def_completion_set_index)

### Reading

#### Enter the set index and the word list here

In [ ]:
reading_start_idx = 1
reading_end_idx = 15
reading_set_index = 1
reading_word_list = tomorrow_review[reading_start_idx-1:reading_end_idx]
reading_exercise = factory.create_exercise(exercise_type='Reading', word_list=reading_word_list)

#### Paste the initial passage here

In [ ]:
initial_passage = pyperclip.paste()
reading_exercise.create_prompt(passage=initial_passage)
reading_exercise.get_prompt()

#### Paste the final passage here

In [ ]:
final_passage = pyperclip.paste()

reading_exercise.import_passage(text=final_passage)

#### Paste the multiple choice questions here

In [ ]:
multiple_choice = pyperclip.paste()
reading_exercise.import_exercise(text=multiple_choice)

##### Paste the solution here

In [ ]:
reading_solution = pyperclip.paste()
reading_exercise.import_solution(text=reading_solution)
reading_exercise.finish_import()
reading_gatherer.import_exercise(exercise=reading_exercise)
reading_exercise_writer.render_template(set_index=reading_set_index)

### Vocabulary exercises

#### Set the word list here and get the prompt to create example sentences

In [24]:
vocab_start_idx = 16
vocab_end_idx = 25
repetition = 'two'

# vocab_word_list = tomorrow_new[vocab_start_idx-1:vocab_end_idx]
vocab_word_list = tomorrow_review[vocab_start_idx-1:vocab_end_idx]

example_sentences = ExampleSentences(vocab_word_list)
example_sentences.create_prompt(number_of_sentences=repetition)
example_sentences.get_prompt()

#### Paste the example sentences here

In [25]:
sentences = pyperclip.paste()
example_sentences.import_sentences(text=sentences)
example_sentences.finish_import()

#### Fill in the gap

##### Enter the set index here

In [26]:
gap_filling_set_index = 6

gap_filling_exercise = factory.create_exercise(exercise_type='Fill in the gap', word_list=example_sentences.word_list, example_sentences=example_sentences)
gap_filling_exercise.finish_import()
gap_filling_gatherer.import_exercise(exercise=gap_filling_exercise)
gap_filling_exercise_writer.render_template(set_index=gap_filling_set_index)

#### Paraphrase

In [27]:
pyperclip.copy(example_sentences.get_second_prompt())

##### Paste the json string here

In [30]:
analyzed_sentences = pyperclip.paste()
example_sentences.import_sentence_analysis(text=analyzed_sentences)

##### Enter the set index here

In [31]:
paraphrase_set_index = 2

paraphrase_exercise = factory.create_exercise(exercise_type='Paraphrase', word_list=example_sentences.word_list, example_sentences=example_sentences)
paraphrase_exercise.finish_import()
paraphrase_gatherer.import_exercise(exercise=paraphrase_exercise)
paraphrase_exercise_writer.render_template(set_index=paraphrase_set_index)

## Finish

In [32]:
configurator.study_n_words(num_of_words_to_learn)